# MuscLiab Notebook for Motion Analysis
In this notebook I want to process the motion files and save them
1) Filter them

2) Normalize

3) Downsample the filtered for use in animation

## Import libraries

In [4]:
import os
import pickle 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

## Import data

In [11]:
# Open dict from pickle
fname = "renick_audience_excerpt_2021-06-14.pickle"
with open(fname, 'rb') as handle: 
    renick_audience_excerpt = pickle.load(handle)

In [6]:
# Open dict from pickle
fname = "khoparzi_audience_excerpt_2021-06-14.pickle"
with open(fname, 'rb') as handle: 
    khoparzi_audience_excerpt = pickle.load(handle)

## Is the sampling rate consistent?
If the sampling rate was consistent the second order difference in the timestamp would be a single number.  
If it isn't, we should resample the time series to make it consistent? Interpolate? Or downsample?  
I am interested in the 0.5-10Hz range therefore I could downsample safely I think.

In [7]:
df_motion = renick_audience_excerpt[1]

In [17]:
# estimate avg sampling rate
freq = int(round(1/ np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
print('sr:', freq, 'Hz')

sr: 60 Hz


In [15]:
diff = np.diff(df_motion.timestamp)
plt.plot(diff)
print('Min: ' + str(min(diff))+ ', Max: ' +str(max(diff))+ ', Mean: '+str(np.mean(diff)))

Min: 7, Max: 36, Mean: 16.66733337037243


This means that the sampling rate varies throughout the dataset. This should not matter so much if I downsample.

### Downsample the signal
The Nyquist criterium determines that your sampling frequency needs to be at least 2x the bandwidth of interest. Finn downsampled to 10Hz.

Maybe could try a spline interpolation?
SciPy digitial signal processign might bw rhw way to go

I emailed finn to inquire how they downsampled. But first: check how the timestamps change with filtering


## 4. Pre-processing the data
### 4.1. Filtering
##### e.g. 4th order, zero-phase IIR lowpass or bandpass filter

In [1]:
from scipy.signal import butter, lfilter

def butter_filt(data, lowcut, highcut, fs, order=4, btype='band'):
    nyq = fs / 2
    b, a = butter(order, [lowcut/nyq, highcut/nyq], btype=btype)
    y = lfilter(b, a, data)
    return 

## Define filter parameters
Finn suggested 0.5-5Hz because 10 Hz is a really high frequency for human motion

In [2]:
lowcut = 0.5
highcut = 5

In [10]:
freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)

ValueError: cannot convert float NaN to integer

In [9]:
renick_filtered = {}
for k,v in renick_audience_excerpt.items():
    df_motion = v
    columns = list(df_motion.iloc[:, 1:4])
    filtered=[]
    freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
    for col in columns:
        filt = butter_filt(df_motion[col], lowcut, highcut, fs=freq, order=4)
        filtered.append(filt)
    renick_filtered[k] = filtered

C:\Users\danasw\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,


ValueError: cannot convert float NaN to integer

In [ ]:
khoparzi_filtered = {}
for k,v in khoparzi_audience.items():
    df_motion = v
    columns = list(df_motion.iloc[:, 1:4])
    filtered=[]
    freq = int(round(1 / np.mean(np.diff(df_motion.timestamp)), 3) * 1000)
    for col in columns:
        filt = butter_filt(df_motion[col], lowcut, highcut, fs=freq, order=4)
        filtered.append(filt)
    khoparzi_filtered[k] = filtered

Finn code

In [ ]:
mob['Time'] = (mob['time']-mob['time'].loc[0]) # Time is time zero 
df = mob 
cols = ['x', 'y', 'z'] 
time_new = np.arange(df['Time'].iat[0], df['Time'].iat[-1], 1/fs) 
df_rs = pd.DataFrame() 
df_rs['Time'] = time_new 
df_rs = df_rs.set_index('Time') 
df_rs['Time (s)'] = time_new/1000 for c in cols: 
    # if not c.startswith('time') and not c.startswith('Time') : 
    f = interpolate.interp1d(df.Time, df[c].values,fill_value='extrapolate') 
    df_rs[c] = f(time_new) 
    mob_rs = df_rs.copy()
